<a href="https://colab.research.google.com/github/soobook/PyTorch-DL/blob/main/code/PT02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2회차: PyTorch 설치 및 기초 코딩

In [ ]:
import torch

print(f'파이토치 버전: {torch.__version__}')
print("GPU" if torch.cuda.is_available() else "CPU")

파이토치 버전: 2.6.0+cu124
GPU


In [ ]:
# torch 및 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# 하이퍼파라미터 설정
batch_size = 64
learning_rate = 0.001
epochs = 5

In [ ]:
# 데이터 전처리 및 로딩
transform = transforms.ToTensor()
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 479kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.83MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.35MB/s]


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# 모델 정의
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.flatten = nn.Flatten()
        self.model = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 모델 생성 및 컴파일 적용
model = DNN().to(device)
compiled_model = torch.compile(model)

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(compiled_model.parameters(), lr=learning_rate)

In [ ]:
# 학습 루프
for epoch in range(epochs): # 전체 에폭 반복
    compiled_model.train() # 학습 모드 설정
    total_loss = 0
    correct = 0
    for data, target in train_loader: # 배치 반복
        data, target = data.to(device), target.to(device) # 입력/라벨 GPU로 이동

        # 순전파 → 손실 → 역전파 → 옵티마이저 스텝
        optimizer.zero_grad()
        output = compiled_model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (output.argmax(1) == target).sum().item()

    # 정확도 계산
    accuracy = 100. * correct / len(train_loader.dataset)
    # 에폭별 결과 출력
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")

W0420 07:00:48.806000 1009 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


Epoch 1, Loss: 316.4419, Accuracy: 90.57%
Epoch 2, Loss: 132.7357, Accuracy: 95.73%
Epoch 3, Loss: 90.1602, Accuracy: 97.05%
Epoch 4, Loss: 67.4648, Accuracy: 97.77%
Epoch 5, Loss: 54.9081, Accuracy: 98.22%


In [ ]:
# 테스트 루프
compiled_model.eval()
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = compiled_model(data)
        correct += (output.argmax(1) == target).sum().item()

test_accuracy = 100. * correct / len(test_loader.dataset)
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 97.41%


### 수업 종료



## 전체 코드

In [ ]:
# torch 및 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 하이퍼파라미터 설정
batch_size = 64
learning_rate = 0.001
epochs = 5

# 데이터 전처리 및 로딩
transform = transforms.ToTensor()
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 정의
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.flatten = nn.Flatten()
        self.model = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 생성 및 컴파일 적용
model = DNN().to(device)
compiled_model = torch.compile(model)

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(compiled_model.parameters(), lr=learning_rate)

# 학습 루프
for epoch in range(epochs): # 전체 에폭 반복
    compiled_model.train() # 학습 모드 설정
    total_loss = 0
    correct = 0
    for data, target in train_loader: # 배치 반복
        data, target = data.to(device), target.to(device) # 입력/라벨 GPU로 이동

        # 순전파 → 손실 → 역전파 → 옵티마이저 스텝
        optimizer.zero_grad()
        output = compiled_model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (output.argmax(1) == target).sum().item()

    # 정확도 계산
    accuracy = 100. * correct / len(train_loader.dataset)
    # 에폭별 결과 출력
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")

# 테스트 루프
compiled_model.eval() # 평가 모드 설정
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = compiled_model(data)
        correct += (output.argmax(1) == target).sum().item()

test_accuracy = 100. * correct / len(test_loader.dataset)
print(f"Test Accuracy: {test_accuracy:.2f}%")


Epoch 1, Loss: 311.5699, Accuracy: 90.70%
Epoch 2, Loss: 126.7101, Accuracy: 95.94%
Epoch 3, Loss: 86.7187, Accuracy: 97.17%
Epoch 4, Loss: 64.7582, Accuracy: 97.80%
Epoch 5, Loss: 51.4485, Accuracy: 98.27%
Test Accuracy: 97.63%


### 종료